# Lấy dữ liệu và chia Train Test 

In [1]:
#Các bước chính:
# 1.Thực hiện đọc dữ liệu và chia train test( đã sửa đổi từ lấy 2600 dữ liệu -> lấy toàn bộ)
# 2. Thực hiện chuyển hóa dữ liệu sang dạng biến liên tục TFIDF
# 3. Thực hiện tính kỳ vọng và phương sai tương ứng 
# 4. Từ bước 3 thực hiện tính các xác suất cần thiết P(c), P(x|c) bằng phân phối Gaussian
# 5. Thực hiện predict và hiển thị kết quả

In [2]:
import numpy as np
import pandas as pd
df = pd.read_csv("D:\\Project\\AI\\EmailsClean.csv")
#khi đọc file có thêm một cột không mong muốn: thực hiện sửa bảng để được data phù hợp : cột email và cột spam
df['email'] = df['emailClean'].apply(lambda x: x.split(" "))
df.columns = ['index','spam','emailClean','email']
df.drop(columns = ['index','emailClean'],inplace = True)
#thực hiện chia train test ,train = 70%data
spam = df[df['spam'] == 1]
notSpam = df[df['spam'] == 0]
l_train_spam = int(len(spam)*0.7)
l_train_notSpam = int(len(notSpam)*0.7)
# l_train_spam = 900
# l_train_notSpam = 900
X_train = spam[0:l_train_spam].append(notSpam[0:l_train_notSpam],ignore_index = True)
X_test = spam[l_train_spam:len(spam)].append(notSpam[l_train_notSpam:len(notSpam)],ignore_index = True)

# Chuyển đổi văn bản sang TFIDF 

In [3]:
X_train

,spam,email
0,1,"[natur, irresist, corpor, ident, lt, realli, h..."
1,1,"[stock, trade, gunsling, fanni, merril, muzo, ..."
2,1,"[nbeliev, new, home, made, easi, im, want, sho..."
3,1,"[color, print, special, request, addit, inform..."
4,1,"[money, get, softwar, cds, softwar, compat, gr..."
...,...,...
3979,0,"[nron, imag, video, enron, imag, video, avail,..."
3980,0,"[interview, steve, submit, evalu, general, imp..."
3981,0,"[subscrib, frbnyrmagl, list, mon, sep, subscri..."
3982,0,"[look, forward, hearn, vinc, pleas, let, know,..."


In [4]:
# create Bag of Word full
BoW = set()
for doc in X_train.email:
    for word in doc:
        BoW.add(word)
len(BoW)

21708

In [5]:
#tính IDF
import math
IDF = dict.fromkeys(BoW,0)
D = len(X_train)
for i in range(0,D):
     for word in set(X_train.email[i]):
            IDF[word]+=1
for word,d in IDF.items():
    IDF[word] = math.log10(float(D/d))
IDF

{'shapiro': 2.8221680793680175,
 'cla': 3.29928933408768,
 'selena': 3.600319329751661,
 'parquet': 3.600319329751661,
 'barron': 3.600319329751661,
 'seamless': 2.646076820312336,
 'bigprofithomebiz': 3.600319329751661,
 'scanner': 2.6972293427597176,
 'lmas': 3.600319329751661,
 'viejo': 3.600319329751661,
 'bic': 3.600319329751661,
 'enthral': 3.600319329751661,
 'fbfb': 3.600319329751661,
 'pageabl': 3.600319329751661,
 'outon': 3.600319329751661,
 'maret': 3.29928933408768,
 'daten': 3.29928933408768,
 'braek': 3.600319329751661,
 'grate': 2.1689555655926736,
 'lvswinjqgxlx': 3.600319329751661,
 'danusia': 3.29928933408768,
 'azmin': 2.998259338423699,
 'ultrascal': 3.600319329751661,
 'rdyerlaw': 3.1231980750319988,
 'pavel': 2.646076820312336,
 'team': 1.0368382443572504,
 'formul': 2.3698704083733872,
 'pirella': 3.600319329751661,
 'smartmortgageusa': 2.8221680793680175,
 'esaibo': 2.454191294073423,
 'saluda': 3.600319329751661,
 'greenzer': 3.29928933408768,
 'wolni': 3.6003

In [6]:
bayes_matrix = np.zeros((X_train.shape[0],len(BoW)))
tfidf = pd.DataFrame(data = bayes_matrix)
tfidf.columns = list(BoW)
for i in range(0,len(X_train)):
    maximum = 0
    #tinh tf
    for word in X_train.email[i]:
        if word in BoW:
            tfidf[word][i]+=1
            maximum = max(maximum,tfidf[word][i])
    for word in X_train.email[i]:
        if word in BoW:
            tfidf[word][i] = float(tfidf[word][i]/float(maximum))*IDF[word]#tinh tfidf
tfidf#bảng các vector tfidf sau khi chuyển các văn bản

,shapiro,cla,selena,parquet,barron,seamless,bigprofithomebiz,scanner,lmas,viejo,...,llaji,monopol,datenanforderungen,norberto,affecteus,beprocess,ltl,busi,yuan,imaq
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.068669,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3979,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.092648,0.0,0.0
3980,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3981,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3982,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


# Tính means và std cho phân phối Gau 

In [7]:
tfidf1 = tfidf.loc[0:l_train_spam-1] #tfidf của các văn bản spam
tfidf0 = tfidf.loc[l_train_spam:len(tfidf)]

In [8]:
#means, std cho spam email
means_spam = pd.DataFrame(data = tfidf1.apply(np.mean,axis = 0))
std_spam = pd.DataFrame(data = tfidf1.apply(np.std,axis = 0))
#means, std cho not spam email
means_notSpam = pd.DataFrame(data = tfidf0.apply(np.mean,axis = 0))
std_notSpam = pd.DataFrame(data = tfidf0.apply(np.std,axis = 0))

In [13]:
#hàm tính P(xi|c) bằng phân phối Gaussian
def norm(x, mean, std):
    #trường hợp std quá nhỏ dẫn tới P(xi|c) xấp xỉ 0 nên sử dụng threshold
    if std <10**-0.774:
        std = 10**-0.774 
    var = float(std)**2
    denom = float((2*math.pi*var)**0.5)
    num = math.exp(-(float(x)-float(mean))**2/(2.0*var))
    return float(num/denom)

In [14]:
P_spam = float((l_train_spam)/(l_train_spam+l_train_notSpam))
P_notSpam = float((l_train_notSpam)/(l_train_spam+l_train_notSpam))
P_spam

0.24021084337349397

# Predict

In [15]:
def predict(doc):
    P_1 = 0.0
    P_0 = 0.0
    #tinh tf-idf của thư mới
    maximum = 0
    tfidf2 = dict.fromkeys(set(word for word in doc),0)
    for word in doc: 
            tfidf2[word]+=1 #tần suất xuất hiện
            maximum = max(maximum,tfidf2[word])
    for word in doc:
        #nếu thư mới có một từ hoàn toàn mới: không xét tới 
        if word in BoW:
            tfidf2[word] = float(tfidf2[word]/float(maximum))*IDF[word]
    for word in doc:
        if word in BoW:
            P_1 += math.log10(norm(tfidf2[word], float(means_spam.loc[word,:]), float(std_spam.loc[word,:])))
            P_0 += math.log10(norm(tfidf2[word], float(means_notSpam.loc[word,:]), float(std_notSpam.loc[word,:])))
    P_1 += math.log10(P_spam)
    P_0 += math.log10(P_notSpam)
    if P_0>P_1:
        return 0
    else:
        return 1

In [16]:
from sklearn.metrics import accuracy_score
pred = pd.DataFrame(data = [predict(d) for d in X_test.email])
accuracy_score(X_test.spam, pred)

0.9286132241076653

In [17]:
from sklearn.metrics import confusion_matrix
cnf_matrix = confusion_matrix(pred,X_test.spam)
print('Confusion matrix:')
print(cnf_matrix)

Confusion matrix:
[[1242   66]
 [  56  345]]
